# Social Cost of Commuting Convenience

These instructions should ensure we have the same environment and datafiles.

* Ensure you have the latest environment from the [git repository](https://github.com/rohancalum/social-cost-convenience/tree/master/docs).
* Ensure you have latest version of [OpenTripPlanner directory](https://drive.google.com/drive/folders/1dpiakMyGko6VMJlpvm_1ocqv_2PX69bZ?usp=sharing) locally. 
* In a new terminal, navigate to the OpenTripPlanner directory. 
* Launch OTP server: `java -Xmx2G -jar otp-1.3.0-shaded.jar --build . --inMemory`
* Activate environment `conda activate geoPython3`
* Restart Jupyter kernel if need be. 

In [8]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))

In [9]:
from src import download, process, router 

In [17]:
cd '/Users/rohannuttall/Desktop/src/social-cost-convenience/src'

/Users/rohannuttall/Desktop/src/social-cost-convenience/src


In [19]:
# For some reason, OTP is not working....

In [18]:
!python router.py

0       47.71791781511426,-122.33253160976562
1      47.606231773346394,-122.33924001169345
2       47.68509616822083,-122.37404441604501
3       47.69652155278548,-122.36394815485704
4       47.68695093426059,-122.39641076534267
                        ...                  
100     47.67719159122908,-122.32420521724731
101     47.68508761549853,-122.36064664456066
102       47.6539147051427,-122.3018653504803
103     47.61998870072279,-122.33351631094172
104     47.65519432029159,-122.38558705901774
Name: coord, Length: 105, dtype: object
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or 

No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be

No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be

No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be

No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be

...

No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be

No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be

No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be

No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be

No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.
No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be